In [16]:
!pip install -q openai-whisper python-dotenv langchain chromadb tiktoken ffmpeg-python

import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("API key loaded successfully.")
else:
    print("Failed to load API key.")



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
API key loaded successfully.


In [10]:
!sudo apt update && sudo apt install -y ffmpeg


Get:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3632 B]
Get:2 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]                 
Get:3 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main amd64 Packages [354 kB]
Get:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease [3961 B]0m
Get:5 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main all Packages [2938 B]
Get:6 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:7 https://dl.yarnpkg.com/debian stable/main amd64 Packages [11.8 kB]       
Get:8 https://dl.yarnpkg.com/debian stable/main all Packages [11.8 kB]        m
Get:9 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Get:10 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable/main amd64 Packages [4557 B]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]0m
Get:13 http://security.ubuntu.com/ubuntu f

In [12]:
!which ffmpeg
!which ffprobe


/usr/bin/ffmpeg
/usr/bin/ffprobe


In [13]:
import yt_dlp

def download_audio(youtube_url, output_path="audio.mp3"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,
        'quiet': True,
        'ffmpeg_location': '/usr/bin',
        'cookiefile': '/workspaces/youtube-rag-searcher/cookieeee.txt',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    return output_path

audio_path = download_audio("https://www.youtube.com/watch?v=3foYyPDp0Ho")
print("🎧 Audio downloaded to:", audio_path)


🎧 Audio downloaded to: audio.mp3


In [17]:
import whisper

model = whisper.load_model("base")
result = model.transcribe(audio_path)
transcript = result['text']

print("📝 Transcript (first 300 chars):\n", transcript[:300])


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 125MiB/s]
/usr/local/python/3.12.1/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📝 Transcript (first 300 chars):
  The time of year has come for the summer of math exposition. So this is a contest that my friend James Schloss and I started back in 2021. The idea is that there's a lot of people out there who have had at least vague thoughts of making some kind of online math explainer. Maybe that's an article, m


In [18]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(transcript)

print(f"📚 Split into {len(chunks)} chunks")


📚 Split into 1 chunks


In [20]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [22]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 23.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [25]:
!sudo apt-get update
!sudo apt-get install -y sqlite3 libsqlite3-dev


Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:4 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]                 
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]      
Hit:6 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease            
Fetched 401 kB in 1s (426 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libsqlite3-0
Suggested packages:
  sqlite3-doc
The following packages will be upgraded:
  libsqlite3-0 libsqlite3-dev sqlite3
3 upgraded, 0 newly installed, 0 to rem

In [34]:
!pip install faiss-cpu



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.7 MB/s eta 0:00:000m eta 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [35]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

docs = [Document(page_content=chunk) for chunk in chunks]
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectordb = FAISS.from_documents(docs, embedding)
print("Embeddings stored in FAISS")

Embeddings stored in FAISS


In [36]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(openai_api_key=openai_api_key),
    retriever=vectordb.as_retriever(),
    return_source_documents=True
)

query = "What is this video about?"
result = qa(query)

print("🤖 Answer:", result['result'])


/tmp/ipykernel_3800/2595056046.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(openai_api_key=openai_api_key),
/tmp/ipykernel_3800/2595056046.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(query)


🤖 Answer: This video is about a math exposition contest called "some four" organized by Grant Sanderson and his friend James Schloss. The contest encourages people to create online math explainers in various formats and topics, aiming to be helpful to students in classrooms. The video explains the rules, deadlines, peer review process, and the special focus on creating content that is beneficial for students and inviting teachers to participate in the peer review process. Grant Sanderson also mentions cash prizes for the winners, sponsored by Brilliant, and emphasizes the importance of creating content that is accessible and useful for a broader audience beyond just math enthusiasts.
